# Transformação e tratamento


## Objetivo: Limpeza e tratamento



- Seleção de colunas

- Tratamento de nulos

- Conversões de tipos

- Outliers

- Feature engineering

- Salvamento em data/processed/

## Introdução

### Transformação e Tratamento dos Dados (Transform)

Este notebook corresponde à segunda etapa do processo ETL (Extract, Transform, Load) e tem como objetivo realizar o tratamento dos dados brutos extraídos do portal Inside Airbnb.

Nesta fase, cada dataset é tratado separadamente, respeitando sua natureza e finalidade, garantindo qualidade, consistência e preparação adequada para a análise exploratória.


## Objetivos da Etapa de Transformação

Os principais objetivos desta etapa são:

- Avaliar a qualidade dos dados brutos
- Tratar valores ausentes e inconsistências
- Padronizar tipos e categorias
- Selecionar variáveis relevantes para análise
- Criar datasets tratados e consistentes
- Preservar rastreabilidade entre dados brutos e processados


## Importação das Bibliotecas

In [1]:
# importação das bibliotecas

import pandas as pd 
import numpy as np 
import os
import geopandas as gpd

## Carregamento dos Dados Brutos

In [2]:
base_path = "../data/raw/"

df_listings_raw = pd.read_csv(base_path + "listings.csv.gz")
df_neighbourhoods_raw = pd.read_csv(base_path + "neighbourhoods.csv")
df_neighbourhoods_geo_raw = gpd.read_file(base_path + "neighbourhoods.geojson")
df_listings = pd.read_csv(base_path + "listings.csv")

## Tratamento do Dataset listings

In [3]:
# leitura dos dados brutos em detalhe
df_listings_raw.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,958,https://www.airbnb.com/rooms/958,20251204025409,2025-12-04,city scrape,"Bright, Modern Garden Unit - 1BR/1BTH",Our bright garden unit overlooks a lovely back...,Quiet cul de sac in friendly neighborhood<br /...,https://a0.muscache.com/pictures/be1bf5ac-a955...,1169,...,4.90,4.98,4.77,STR-0006854,f,1,1,0,0,2.54
1,5858,https://www.airbnb.com/rooms/5858,20251204025409,2025-12-04,city scrape,Creative Sanctuary,We live in a large Victorian house on a quiet ...,I love how our neighborhood feels quiet but is...,https://a0.muscache.com/pictures/hosting/Hosti...,8904,...,4.85,4.77,4.68,NaN,f,1,1,0,0,0.52
2,8142,https://www.airbnb.com/rooms/8142,20251204025409,2025-12-04,city scrape,*FriendlyRoom Apt. Style -UCSF/USF - San Franc...,Nice and good public transportation. 7 minute...,"N Juda Muni, Bus and UCSF Shuttle.<br /><br />...",https://a0.muscache.com/pictures/hosting/Hosti...,21994,...,4.80,4.70,4.70,NaN,f,20,0,20,0,0.07
3,8339,https://www.airbnb.com/rooms/8339,20251204025409,2025-12-04,city scrape,Historic Alamo Square Victorian,"For creative humans who love art, space, photo...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,24215,...,5.00,4.94,4.75,STR-0000264,f,1,1,0,0,0.13
4,10537,https://www.airbnb.com/rooms/10537,20251204025409,2025-12-04,city scrape,Elegant & Cozy w/City views. Private room: Purple,Casa de Paz (House of Peace) is like staying w...,NaN,https://a0.muscache.com/pictures/airflow/Hosti...,36752,...,5.00,4.68,4.80,2022-011003STR,f,3,1,2,0,0.24


In [4]:
df_listings_raw.shape

(7535, 85)

In [5]:
df_listings_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7535 entries, 0 to 7534
Data columns (total 85 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            7535 non-null   int64  
 1   listing_url                                   7535 non-null   object 
 2   scrape_id                                     7535 non-null   int64  
 3   last_scraped                                  7535 non-null   object 
 4   source                                        7535 non-null   object 
 5   name                                          7535 non-null   object 
 6   description                                   7445 non-null   object 
 7   neighborhood_overview                         4168 non-null   object 
 8   picture_url                                   7535 non-null   object 
 9   host_id                                       7535 non-null   i

In [6]:
df_listings_raw.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_profile_id', 'host_profile_url', 'host_name',
       'host_since', 'hosts_time_as_user_years', 'hosts_time_as_user_months',
       'hosts_time_as_host_years', 'hosts_time_as_host_months',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'latitude', 'longitude',
       'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minim

In [7]:
# informações iniciais dos dados resumidos 
df_listings.head()

,id,name,host_id,host_profile_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",1169,1462506189282101689,Holly,NaN,Western Addition,37.77028,-122.43317,Entire home/apt,NaN,2,507,2025-11-15,2.54,1,228,41,STR-0006854
1,5858,Creative Sanctuary,8904,1462506623299518225,Philip Jonathon,NaN,Bernal Heights,37.74474,-122.42089,Entire home/apt,NaN,30,105,2017-08-06,0.52,1,365,0,NaN
2,8142,*FriendlyRoom Apt. Style -UCSF/USF - San Franc...,21994,1462506956810615042,Aaron,NaN,Haight Ashbury,37.76555,-122.45213,Private room,NaN,32,10,2023-07-30,0.07,20,362,0,NaN
3,8339,Historic Alamo Square Victorian,24215,1462506994551169471,Rosmarie,NaN,Western Addition,37.77377,-122.43614,Entire home/apt,NaN,9,25,2019-06-28,0.13,1,339,0,STR-0000264
4,10537,Elegant & Cozy w/City views. Private room: Purple,36752,1462507288958203289,Teresa,NaN,Bayview,37.71750,-122.39698,Private room,NaN,1,46,2025-11-07,0.24,3,365,12,2022-011003STR


In [8]:
df_listings.shape

(7535, 19)

In [9]:
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7535 entries, 0 to 7534
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              7535 non-null   int64  
 1   name                            7535 non-null   object 
 2   host_id                         7535 non-null   int64  
 3   host_profile_id                 7535 non-null   int64  
 4   host_name                       7341 non-null   object 
 5   neighbourhood_group             0 non-null      float64
 6   neighbourhood                   7535 non-null   object 
 7   latitude                        7535 non-null   float64
 8   longitude                       7535 non-null   float64
 9   room_type                       7535 non-null   object 
 10  price                           0 non-null      float64
 11  minimum_nights                  7535 non-null   int64  
 12  number_of_reviews               75

In [10]:
df_listings.columns

Index(['id', 'name', 'host_id', 'host_profile_id', 'host_name',
       'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude',
       'room_type', 'price', 'minimum_nights', 'number_of_reviews',
       'last_review', 'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license'],
      dtype='object')

## Tratamento do Dataset listings

In [11]:
listings_cols = [
    'id',
    'name',
    'host_id',
    'host_name',
    'neighbourhood', 
    'latitude', 
    'longitude',
    'property_type', 
    'room_type', 
    'accommodates', 
    'bathrooms',
    'bathrooms_text', 
    'bedrooms', 
    'beds', 
    'minimum_nights', 
    'maximum_nights', 
    'has_availability',
    'availability_30', 
    'availability_60', 
    'availability_90',
    'availability_365', 
    'number_of_reviews',
    'first_review', 
    'last_review',
    'review_scores_rating', 
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin', 
    'review_scores_location',
    'review_scores_value', 
    'license', 
    'instant_bookable',
    'calculated_host_listings_count',
    'reviews_per_month'
]

df = df_listings_raw[listings_cols].copy()

**As colunas `bathrooms` e `bathrooms_text` foram mantidas temporariamente
durante a etapa de transformação para permitir a validação e imputação
dos valores ausentes de banheiros. Após o tratamento, a coluna
`bathrooms_text` foi removida por redundância.**

In [12]:
# verificando as colunas selecionadas
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'minimum_nights',
       'maximum_nights', 'has_availability', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_location', 'review_scores_value', 'license',
       'instant_bookable', 'calculated_host_listings_count',
       'reviews_per_month'],
      dtype='object')

In [13]:
# verificando o tipo de dado
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7535 entries, 0 to 7534
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              7535 non-null   int64  
 1   name                            7535 non-null   object 
 2   host_id                         7535 non-null   int64  
 3   host_name                       7341 non-null   object 
 4   neighbourhood                   4168 non-null   object 
 5   latitude                        7535 non-null   float64
 6   longitude                       7535 non-null   float64
 7   property_type                   7535 non-null   object 
 8   room_type                       7535 non-null   object 
 9   accommodates                    7535 non-null   int64  
 10  bathrooms                       5727 non-null   float64
 11  bathrooms_text                  7476 non-null   object 
 12  bedrooms                        70

## Significado de cada coluna




### Dicionário de Dados — Dataset listings

- **id** – identificador único do imóvel anunciado na plataforma Airbnb.

- **name** – nome ou título do anúncio da propriedade.

- **host_id** – identificador único do anfitrião (proprietário) do imóvel.

- **host_name** – nome do anfitrião responsável pelo anúncio.

- **neighbourhood** – nome do bairro onde o imóvel está localizado.

- **latitude** – coordenada de latitude do imóvel, utilizada para análises geográficas.

- **longitude** – coordenada de longitude do imóvel, utilizada para análises geográficas.

- **property_type** – tipo da propriedade (ex.: apartamento, casa, quarto em residência, etc.).

- **room_type** – tipo de acomodação ofertada (ex.: imóvel inteiro, quarto privado ou compartilhado).

- **accommodates** – número máximo de pessoas que o imóvel pode acomodar.

- **bathrooms** – número de banheiros disponíveis no imóvel.

- **bathrooms_text** – descrição textual da quantidade e tipo de banheiros (ex.: “1 banheiro privativo”).

- **bedrooms** – número de quartos disponíveis no imóvel.

- **beds** – número total de camas disponíveis para os hóspedes.

- **minimum_nights** – número mínimo de noites exigido para realizar uma reserva.

- **maximum_nights** – número máximo de noites permitidas por reserva.

- **has_availability** – indica se o imóvel possui disponibilidade para reservas no período analisado.

- **availability_30** – número de dias disponíveis para reserva nos próximos 30 dias.

- **availability_60** – número de dias disponíveis para reserva nos próximos 60 dias.

- **availability_90** – número de dias disponíveis para reserva nos próximos 90 dias.

- **availability_365** – número de dias disponíveis para reserva nos próximos 365 dias.

- **number_of_reviews** – quantidade total de avaliações recebidas pelo imóvel.

- **first_review** – data da primeira avaliação registrada para o imóvel.

- **last_review** – data da avaliação mais recente registrada para o imóvel.

- **review_scores_rating** – avaliação geral do imóvel atribuída pelos hóspedes.

- **review_scores_accuracy** – avaliação referente à precisão das informações do anúncio.

- **review_scores_cleanliness** – avaliação relacionada ao nível de limpeza do imóvel.

- **review_scores_checkin** – avaliação da experiência de check-in realizada pelos hóspedes.

- **review_scores_location** – avaliação da localização do imóvel.

- **review_scores_value** – avaliação do custo-benefício percebido pelos hóspedes.

- **license** – informação sobre a licença ou regularização do imóvel para locação, quando disponível.

- **instant_bookable** – indica se o imóvel permite reserva imediata sem necessidade de aprovação do anfitrião.

- **calculated_host_listings_count** – número total de anúncios ativos associados ao mesmo anfitrião.

- **reviews_per_month** – média de avaliações recebidas pelo imóvel por mês.

In [14]:
df.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,property_type,room_type,accommodates,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,reviews_per_month
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",1169,Holly,Neighborhood highlights,37.77028,-122.43317,Entire serviced apartment,Entire home/apt,3,...,4.88,4.94,4.93,4.96,4.98,4.77,STR-0006854,f,1,2.54
1,5858,Creative Sanctuary,8904,Philip Jonathon,Neighborhood highlights,37.74474,-122.42089,Entire rental unit,Entire home/apt,4,...,4.87,4.85,4.87,4.89,4.77,4.68,NaN,f,1,0.52
2,8142,*FriendlyRoom Apt. Style -UCSF/USF - San Franc...,21994,Aaron,Neighborhood highlights,37.76555,-122.45213,Private room in rental unit,Private room,1,...,4.70,4.50,4.50,4.80,4.70,4.70,NaN,f,20,0.07
3,8339,Historic Alamo Square Victorian,24215,Rosmarie,NaN,37.77377,-122.43614,Entire condo,Entire home/apt,2,...,4.86,4.88,5.00,4.94,4.94,4.75,STR-0000264,f,1,0.13
4,10537,Elegant & Cozy w/City views. Private room: Purple,36752,Teresa,NaN,37.71750,-122.39698,Private room,Private room,2,...,4.98,4.93,4.98,4.95,4.68,4.80,2022-011003STR,f,3,0.24


In [15]:
# verificar valores NaN (nulos / ausentes)
df.isnull().sum()

id                                   0
name                                 0
host_id                              0
host_name                          194
neighbourhood                     3367
latitude                             0
longitude                            0
property_type                        0
room_type                            0
accommodates                         0
bathrooms                         1808
bathrooms_text                      59
bedrooms                           439
beds                              1843
minimum_nights                       0
maximum_nights                       0
has_availability                   257
availability_30                      0
availability_60                      0
availability_90                      0
availability_365                     0
number_of_reviews                    0
first_review                      1673
last_review                       1673
review_scores_rating              1673
review_scores_accuracy   

In [16]:
# preencher com desconhecido os valores ausente de host_name e 'neighbourhood'
df['host_name'] = df['host_name'].fillna('Unknown')
df['neighbourhood'] = df['neighbourhood'].fillna('Unknown')


### Tratamento de valores ausentes

Os valores ausentes nas colunas `host_name` e `neighbourhood` foram preenchidos
com a categoria `"Unknown"`. Essa decisão foi adotada para preservar os registros
no dataset, evitando a exclusão de anúncios válidos.

No caso de `host_name`, o identificador principal do anfitrião é a coluna
`host_id`, o que garante a rastreabilidade mesmo quando o nome não está
disponível. Para a variável `neighbourhood`, o valor `"Unknown"` indica
explicitamente a ausência de informação geográfica fornecida pela fonte,
permitindo sua inclusão em análises categóricas sem introduzir suposições.


In [17]:
df.isnull().sum()

id                                   0
name                                 0
host_id                              0
host_name                            0
neighbourhood                        0
latitude                             0
longitude                            0
property_type                        0
room_type                            0
accommodates                         0
bathrooms                         1808
bathrooms_text                      59
bedrooms                           439
beds                              1843
minimum_nights                       0
maximum_nights                       0
has_availability                   257
availability_30                      0
availability_60                      0
availability_90                      0
availability_365                     0
number_of_reviews                    0
first_review                      1673
last_review                       1673
review_scores_rating              1673
review_scores_accuracy   

In [18]:
# verificar algumas colunas sobre a caracteritica do imóvel

cols = [
    'property_type', 
    'room_type', 
    'accommodates', 
    'bathrooms',
    'bathrooms_text', 
    'bedrooms', 
    'beds'
]

df_property = df[cols]

In [19]:
df_property.head(10)

,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds
0,Entire serviced apartment,Entire home/apt,3,1.0,1 bath,1.0,2.0
1,Entire rental unit,Entire home/apt,4,2.0,2 baths,2.0,2.0
2,Private room in rental unit,Private room,1,4.0,4 shared baths,3.0,1.0
3,Entire condo,Entire home/apt,2,1.5,1.5 baths,1.0,1.0
4,Private room,Private room,2,1.5,1.5 shared baths,1.0,1.0
5,Entire rental unit,Entire home/apt,2,1.0,1 bath,0.0,1.0
6,Private room in condo,Private room,2,1.0,1 shared bath,1.0,1.0
7,Private room in rental unit,Private room,1,3.0,3 shared baths,1.0,2.0
8,Private room in rental unit,Private room,1,3.0,3 shared baths,1.0,2.0
9,Private room in rental unit,Private room,2,1.5,1.5 shared baths,1.0,1.0


In [20]:
# verificando os valores ausentes de bathrooms_text
df_property.loc[df_property['bathrooms'].isna()]


,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds
21,Private room in rental unit,Private room,1,NaN,1 private bath,NaN,NaN
26,Entire condo,Entire home/apt,2,NaN,1 bath,1.0,NaN
39,Private room in rental unit,Private room,1,NaN,1 shared bath,NaN,NaN
40,Private room in home,Private room,2,NaN,1 bath,1.0,NaN
47,Entire rental unit,Entire home/apt,2,NaN,1 bath,1.0,NaN
...,...,...,...,...,...,...,...
7387,Private room in condo,Private room,2,NaN,1.5 shared baths,1.0,NaN
7402,Entire rental unit,Entire home/apt,10,NaN,2 baths,5.0,NaN
7410,Room in hotel,Private room,4,NaN,1 private bath,NaN,NaN
7415,Room in hotel,Private room,2,NaN,1 private bath,1.0,NaN


In [21]:
# verificando os valores ausentes de bathrooms_text
df_property.loc[df_property['bathrooms_text'].isna()]

,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds
176,Private room in home,Private room,2,NaN,NaN,1.0,1.0
177,Private room in home,Private room,2,NaN,NaN,1.0,1.0
231,Entire rental unit,Entire home/apt,2,NaN,NaN,1.0,1.0
3509,Private room in home,Private room,1,NaN,NaN,1.0,1.0
4216,Room in hotel,Hotel room,16,NaN,NaN,4.0,NaN
4217,Room in hotel,Hotel room,16,NaN,NaN,4.0,NaN
4218,Room in hotel,Hotel room,12,NaN,NaN,3.0,NaN
4219,Room in hotel,Hotel room,12,NaN,NaN,3.0,NaN
4220,Room in hotel,Hotel room,12,NaN,NaN,3.0,NaN
4221,Room in hotel,Hotel room,8,NaN,NaN,2.0,NaN


In [22]:
# Extrarir da variavel bathrooms_text os valores  de bathrooms
df['bathrooms_from_text'] = (
    df['bathrooms_text']
    .str.extract(r'(\d+\.?\d*)')
    .astype(float)
)

df['bathrooms'] = df['bathrooms'].fillna(df['bathrooms_from_text'])

In [23]:
df.isnull().sum()

id                                   0
name                                 0
host_id                              0
host_name                            0
neighbourhood                        0
latitude                             0
longitude                            0
property_type                        0
room_type                            0
accommodates                         0
bathrooms                           51
bathrooms_text                      59
bedrooms                           439
beds                              1843
minimum_nights                       0
maximum_nights                       0
has_availability                   257
availability_30                      0
availability_60                      0
availability_90                      0
availability_365                     0
number_of_reviews                    0
first_review                      1673
last_review                       1673
review_scores_rating              1673
review_scores_accuracy   

In [24]:
# inputar o valor de 1 se accommodates == 1
mask = df['bathrooms'].isna() & (df['accommodates'] == 1)
df.loc[mask, 'bathrooms'] = 1

In [25]:
# Regra 3: fallback por room_type
df['bathrooms'] = df.groupby('room_type')['bathrooms'].transform(
    lambda x: x.fillna(x.median())
)

In [26]:
# remover a coluna bathrooms_text
cols_to_drop = ['bathrooms_from_text', 'bathrooms_text']
df.drop(columns=cols_to_drop, inplace=True)

#### Tratamento de valores ausentes — variável bathrooms

A variável bathrooms apresentou uma quantidade relevante de valores ausentes. Para lidar com esse problema, foi adotada uma estratégia de imputação progressiva, priorizando informações mais específicas e reduzindo o risco de introdução de viés artificial nos dados.

O tratamento foi realizado em três etapas sequenciais:

1️⃣ Extração a partir da variável bathrooms_text

Inicialmente, os valores ausentes em bathrooms foram preenchidos a partir da variável textual bathrooms_text, que contém descrições como “1 bath”, “1.5 baths” ou “Shared bath”.
Foi aplicada uma rotina de extração numérica para recuperar a quantidade de banheiros sempre que essa informação estivesse explicitamente disponível no texto.

Essa etapa permitiu recuperar valores reais informados pelos anfitriões, preservando a fidelidade semântica dos dados originais.

2️⃣ Imputação baseada na capacidade de acomodação (accommodates)

Na sequência, para os registros que ainda permaneceram com valores ausentes, foi aplicada a seguinte regra:

Quando accommodates == 1, o valor de bathrooms foi imputado como 1.

Essa decisão se baseia no padrão observado em anúncios do Airbnb, onde acomodações destinadas a uma única pessoa geralmente dispõem de ao menos um banheiro (privativo ou compartilhado).
Essa regra representa uma imputação conservadora e plausível, evitando valores nulos em casos simples e frequentes.

3️⃣ Fallback por tipo de acomodação (room_type) usando a mediana

Por fim, para os casos residuais em que ainda não havia informação disponível, foi adotada uma estratégia de fallback baseada no room_type.
Nessa etapa, os valores ausentes de bathrooms foram preenchidos utilizando a mediana do número de banheiros dentro de cada categoria de tipo de quarto.

A escolha da mediana — em vez da média — visa reduzir a influência de outliers e preservar a robustez estatística da imputação, respeitando diferenças estruturais entre tipos de acomodação, como Entire home/apt, Private room e Shared room.

In [28]:
cols = [
    'property_type', 
    'room_type', 
    'accommodates', 
    'bathrooms',
    'bedrooms', 
    'beds'
]

data_property = df[cols]

In [33]:
data_property.head(10)

,property_type,room_type,accommodates,bathrooms,bedrooms,beds
0,Entire serviced apartment,Entire home/apt,3,1.0,1.0,2.0
1,Entire rental unit,Entire home/apt,4,2.0,2.0,2.0
2,Private room in rental unit,Private room,1,4.0,3.0,1.0
3,Entire condo,Entire home/apt,2,1.5,1.0,1.0
4,Private room,Private room,2,1.5,1.0,1.0
5,Entire rental unit,Entire home/apt,2,1.0,0.0,1.0
6,Private room in condo,Private room,2,1.0,1.0,1.0
7,Private room in rental unit,Private room,1,3.0,1.0,2.0
8,Private room in rental unit,Private room,1,3.0,1.0,2.0
9,Private room in rental unit,Private room,2,1.5,1.0,1.0


In [32]:
# verificando os valores ausentes de bathrooms_text
data_property.loc[data_property['bedrooms'].isna()]

,property_type,room_type,accommodates,bathrooms,bedrooms,beds
21,Private room in rental unit,Private room,1,1.0,NaN,NaN
39,Private room in rental unit,Private room,1,1.0,NaN,NaN
59,Private room in home,Private room,1,1.0,NaN,NaN
68,Private room in rental unit,Private room,1,1.5,NaN,NaN
92,Private room in rental unit,Private room,2,1.5,NaN,NaN
...,...,...,...,...,...,...
7162,Private room in home,Private room,2,2.0,NaN,NaN
7250,Entire rental unit,Entire home/apt,4,1.0,NaN,NaN
7281,Private room in home,Private room,1,1.5,NaN,NaN
7294,Room in hotel,Private room,4,1.0,NaN,NaN


In [34]:
# inputar o valor 1 onde se tem 1 a accommodates tanto para bedrooms beds
mask = df['bedrooms'].isna() & (df['accommodates'] == 1)
df.loc[mask, 'bedrooms'] = 1


In [35]:
mask = df['beds'].isna() & (df['accommodates'] == 1)
df.loc[mask, 'beds'] = 1

In [37]:
cols = [
    'property_type', 
    'room_type', 
    'accommodates', 
    'bathrooms',
    'bedrooms', 
    'beds'
]

data_property = df[cols]

In [38]:
data_property.head(10)

,property_type,room_type,accommodates,bathrooms,bedrooms,beds
0,Entire serviced apartment,Entire home/apt,3,1.0,1.0,2.0
1,Entire rental unit,Entire home/apt,4,2.0,2.0,2.0
2,Private room in rental unit,Private room,1,4.0,3.0,1.0
3,Entire condo,Entire home/apt,2,1.5,1.0,1.0
4,Private room,Private room,2,1.5,1.0,1.0
5,Entire rental unit,Entire home/apt,2,1.0,0.0,1.0
6,Private room in condo,Private room,2,1.0,1.0,1.0
7,Private room in rental unit,Private room,1,3.0,1.0,2.0
8,Private room in rental unit,Private room,1,3.0,1.0,2.0
9,Private room in rental unit,Private room,2,1.5,1.0,1.0


In [39]:
# verificando os valores ausentes de bathrooms_text
data_property.loc[data_property['bedrooms'].isna()]

,property_type,room_type,accommodates,bathrooms,bedrooms,beds
92,Private room in rental unit,Private room,2,1.5,NaN,NaN
115,Entire rental unit,Entire home/apt,2,1.0,NaN,1.0
181,Private room in cottage,Private room,2,1.0,NaN,NaN
249,Entire condo,Entire home/apt,2,1.0,NaN,NaN
260,Private room in rental unit,Private room,2,1.0,NaN,NaN
...,...,...,...,...,...,...
7161,Room in hotel,Private room,2,1.0,NaN,NaN
7162,Private room in home,Private room,2,2.0,NaN,NaN
7250,Entire rental unit,Entire home/apt,4,1.0,NaN,NaN
7294,Room in hotel,Private room,4,1.0,NaN,NaN


In [40]:
df['bedrooms'] = df.groupby('room_type')['bedrooms'].transform(
    lambda x: x.fillna(x.median())
)

In [42]:
df['beds'] = df.groupby('room_type')['beds'].transform(
    lambda x: x.fillna(x.median())
)

#### Tratamento de valores ausentes — variável bedrooms e beds

A variável bathrooms apresentou uma quantidade relevante de valores ausentes. Para lidar com esse problema, foi adotada uma estratégia de imputação progressiva, priorizando informações mais específicas e reduzindo o risco de introdução de viés artificial nos dados.

O tratamento foi realizado em duas etapas sequenciais:

1️⃣ Imputação baseada na capacidade de acomodação (accommodates)

Na sequência, para os registros que ainda permaneceram com valores ausentes, foi aplicada a seguinte regra:

Quando accommodates == 1, o valor de bedrooms e beds foi imputado como 1.

Essa decisão se baseia no padrão observado em anúncios do Airbnb, onde acomodações destinadas a uma única pessoa geralmente dispõem de ao menos um quarto e uma cama (privativo ou compartilhado).
Essa regra representa uma imputação conservadora e plausível, evitando valores nulos em casos simples e frequentes.

2️⃣ Fallback por tipo de acomodação (room_type) usando a mediana

Por fim, para os casos residuais em que ainda não havia informação disponível, foi adotada uma estratégia de fallback baseada no room_type.
Nessa etapa, os valores ausentes de bedrooms e beds foram preenchidos utilizando a mediana do número de banheiros dentro de cada categoria de tipo de quarto.

A escolha da mediana — em vez da média — visa reduzir a influência de outliers e preservar a robustez estatística da imputação, respeitando diferenças estruturais entre tipos de acomodação, como Entire home/apt, Private room e Shared room.


In [46]:
cols = [
    'has_availability', 
    'availability_30', 
    'availability_60', 
    'availability_90',
    'availability_365'
]

data_availability = df[cols]

In [48]:
data_availability.head()

,has_availability,availability_30,availability_60,availability_90,availability_365
0,t,4,10,15,228
1,t,30,60,90,365
2,t,27,57,87,362
3,t,4,34,64,339
4,t,30,60,90,365


In [45]:
# tratamento da variavel has_availability (257 ausentes)
df['has_availability'] = df['has_availability'].fillna('f')



### O que representa

Indica se o anúncio possui disponibilidade ativa no calendário (t / f).

✅ Melhor tratamento

Preencher com 'f' (False)

📌 Justificativa

O dataset já possui as colunas availability_30, availability_60, availability_90 e availability_365

Se todas essas colunas estão presentes, mas has_availability está ausente, o caso mais conservador é assumir não disponível

Evita inflar artificialmente a oferta


In [50]:
## mantido reviews e datas nans e preenchido com 0 reviews_per_month
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

In [52]:
# preencher license com not provided
df['license'] = df['license'].fillna('Not provided')

In [53]:
df.isnull().sum()

id                                   0
name                                 0
host_id                              0
host_name                            0
neighbourhood                        0
latitude                             0
longitude                            0
property_type                        0
room_type                            0
accommodates                         0
bathrooms                            0
bedrooms                             0
beds                                 0
minimum_nights                       0
maximum_nights                       0
has_availability                     0
availability_30                      0
availability_60                      0
availability_90                      0
availability_365                     0
number_of_reviews                    0
first_review                      1673
last_review                       1673
review_scores_rating              1673
review_scores_accuracy            1673
review_scores_cleanliness

**Os valores ausentes relacionados a reviews foram mantidos quando a ausência representava falta de avaliação real, evitando imputação artificial de notas. Para variáveis operacionais e categóricas, foram aplicadas imputações conservadoras, priorizando coerência semântica e redução de viés analítico.**

## Salvar o dataset transformado

In [54]:
# Definir caminho
processed_path = "../data/processed/"

# Salvar em Parquet
df.to_parquet(processed_path + "listings_sanfrancisco_v1.parquet", index=False)

# Opcional: salvar CSV também
df.to_csv(processed_path + "listings_sanfrancisco_v1.csv", index=False)

## Proximos passos

- Use este dataset limpo e padronizado nas próximas etapas:

- EDA (03_eda.ipynb)

- Modelagem / análise estatística

- Visualizações e dashboards